In [ ]:
import numpy as np
import plotly
import json
import os
import plotly.express as px
import numpy as np
import statistics as st
import pandas as pd



def write_to_csv(data, name):
    '''
    data in dict format, where keys form the column names
    '''
    df = pd.DataFrame(data)
    df.to_csv(name+'.csv', index=False)


########### trace processing ##########
def read_traces(log_path):
    '''
    read the trace files and extract variable names
    data = [ [event, timestamp], [], [],......,[] ]
    '''
    with open(log_path, 'r') as f:
        data = json.load(f)
    return data


def get_uniquevar(raw_trace):
    ''' 
    convert the v2.2 trace into list of unique variables
    raw_trace = data from read_traces, list( (var, ts),(var, ts),(var, ts),.... )
    return:
        unique_var = list(var1,var2,...) ## list of strings
    '''
    unique_var = []
    for rt in raw_trace:
        [var, timestamp] = rt
        # print([var, timestamp])
        if var not in unique_var:
            unique_var += [var]
            # print(rt)
    return unique_var


def generate_map(unique_events):
    '''
    unique_events -> list of all the variables in the code (unique, and in order of logging)
    return:
        event_map -> takes the variable name and gives corresponding event number
        event_remap -> takes event number and gives associated variable name
    '''
    event_map = dict()
    event_remap = dict()
    for i in range(len(unique_events)):
        event_remap[i+1] = unique_events[i]
        event_map[unique_events[i]] = i+1

    return(event_map, event_remap)


In [ ]:
############ configuration ################
############################################

code = 'theft_protection'       ### application (code)
behaviour = 'faulty_data'            ### normal, faulty_data
thread_typ = 'single'           ### single, multi
version = 2.2                     ### format of data collection

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{code}/{thread_typ}_thread/version_{version}/{behaviour}'

### Get paths to the files

In [ ]:
###### get file paths #######

all_files = os.listdir(log_path)
all_files.sort()
logs = []
traces = []
unknown = []
for i in all_files:
    if i.find('log') == 0:
        logs += [i]
    elif i.find('trace') == 0 and i.find('.txt') == -1:
        traces += [i]
    else:
        unknown += [i]

######### path to files
paths_log = [os.path.join(log_path, x) for x in logs]
paths_traces = [os.path.join(log_path, x) for x in traces]
paths_log.sort()

In [ ]:
### get variable list that is same for all log files ####
raw_trace = read_traces(paths_traces[0])
_var_list = get_uniquevar(raw_trace)
np.save('var_list.npy', _var_list, allow_pickle=False)
to_number, from_number = generate_map(_var_list)


In [ ]:
print(paths_traces)

### Process Traces

In [ ]:
########## process the traces ###########

col_data = []
for (p,w) in zip(paths_traces, traces):
    trace = read_traces(p)
    num_trace = []
    time_stamp = []
    for (t, ts) in trace:
        nt = to_number[t]
        num_trace.extend([nt])
        time_stamp.extend([ts])
        # ### take limited samples
        # if ts > 250000:
        #     break
    col_data += [(w, time_stamp, num_trace, _var_list, p)]   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

In [ ]:
col_data[2]

### Generate plot trace data

In [ ]:
all_df = []
for col in col_data:
    # print(col)
    plot_data = dict()
    plot_data['time'] = col[1]   ### x_data
    plot_data[col[0]] = col[2]   ### y_data (traces)

    ### convert the list to data frame and store it for plotting
    df = pd.DataFrame(plot_data)
    all_df += [df]

    #break


In [ ]:
all_df[2]['time'][720]

### plot data in plotly

In [ ]:
# import plotly.graph_objects as go
# import pandas as pd


# # Create figure
# fig = go.Figure()

# ### add all the traces to the graph
# for df in all_df:
#     df_col = df.columns
#     fig.add_trace(
#                 go.Scatter( x=list(df[df_col[0]]), y=list(df[df_col[1]]))   ### equivalent to: x=list(df['time']), y=list(df['trace1'])
#                 )
    

# # Add range slider, title, yticks, axes labels
# fig.update_layout(
#     title_text="Event Trace Time",
#     xaxis=dict(
#         title="Number of events",
#         rangeslider=dict(visible=True),
#         type='linear'
#     ),
#     yaxis=dict(
#         title="Variables",
#         tickvals=[k for k in range(1,len(_var_list)+1)],
#         ticktext=_var_list
#     ),
#     autosize=True,
#     #width=500,
#     height=600,
    
# )


# # style all the traces
# fig.update_traces(
#     #hoverinfo="name+x+text",
#     line={"width": 0.5},
#     marker={"size": 8},
#     mode="lines+markers",
#     showlegend=True,
# )

# fig.show()


In [ ]:
x_ticks

### Generate excel sheet

In [ ]:
######### write the data to excel sheet ###########

import xlsxwriter

workbook = xlsxwriter.Workbook(f'{thread_typ}_version{version}_eventtrace_{behaviour}.xlsx')

### add worksheet for mapper
worksheet = workbook.add_worksheet('mapper')

### convert dict to list and write to xls

worksheet.write(0, 0, 'event num')
worksheet.write(0, 1, 'event name')

row = 1
col = 0
keym = list(from_number.keys())
for k in keym:
    val = from_number[k]
    worksheet.write(row, col, k)
    worksheet.write(row, col+1, val)
    row+=1

### add worksheet for traces
worksheet = workbook.add_worksheet('traces')
row = 0
col = 0
for (name, time, trace, labels) in col_data:
    worksheet.write(row, col, name)
    row=1
    worksheet.write(row, col, 'events')
    row=2
    for t in trace:
        #worksheet.write(row, col, from_number[t])
        worksheet.write(row, col, t)
        row+=1
    row=1
    col+=1
    worksheet.write(row, col, 'timestamp')
    row=2
    for t in time:
        worksheet.write(row, col, t)
        row+=1

    col+=2
    row=0

workbook.close()

### Variable wise plots

In [ ]:
paths_traces

In [ ]:
from collections import defaultdict 

tl_data = []     ### [log1, log,2, ... ] --> [{var1:[], var2:[], ....}, {}  ]

for (p,w) in zip(paths_traces, traces):
    #print(p,w)
    trace = read_traces(p)
    #print(trace)
    var_timelist = defaultdict(list)
    # for var_name in _var_list:
    #     print(var_name)
    for ind, (t, ts) in enumerate(trace):
        #print(t,ts)
        var_timelist[t] += [[ts, ind]]     ### format: {var1:[[ts, ind], ...], var2:[[ts, ind], ...]}

    tl_data += [var_timelist]    
    #break


In [ ]:
trace

In [ ]:
############ prepare data to plot #############
###############################################

to_plot = []   ### in format -> [var_name, ( [[<exe inters of var in log1>], [timestamps ]  )]

#### use _var_list for consistency

### collect data for each variable from each log file
for v in _var_list:
    xy_data = [] ### execution intervals
    log_names = []
    for i, (p,w, data) in enumerate(zip(paths_log, logs, tl_data)):
        # print('i:',i)

        ##############################
        if i==1:   ### take data of single log for plotting
        #################################

            try:
                #_, data = read_logs(p)   ### data of each log file
                time_list = data[v]   ### get the timestamps
                #print(data)
                exe_time = []
                timestamp = []
                indices = []

                # print(len(data))
                # print(time_list)
        
                for _, (td1,td2) in enumerate(zip(time_list[0:-1], time_list[1:])):
                    t2 = td2[0]
                    t1 = td1[0]
                    ind = td2[1]
                    tdiff = t2-t1
                    #print(tdiff)
                    exe_time+=[tdiff]
                    timestamp+=[t2] 
                    indices+=[ind]  ### index of the event in trace files | to maps events between full trace plots and variable plots
                    
                    # print(ind,t2)
                    # ##### testing limited samples  
                    # if t2 > 250000:
                    #     break
                    # ###### testing limited samples

                print('length of exe_time:', len(exe_time))
                assert(len(exe_time)==len(timestamp))
                log_names += [w]
                xy_data += [(exe_time,timestamp, indices)]
            except Exception as e:
                print(f'{w}:{v}', e)    ### will execute if any variable has only one execution time
        
    assert(len(log_names)==len(xy_data))
    to_plot += [(p.replace(w,v),log_names,xy_data)]  ### [name of the file to write plots(variable name), labels for legend (log names), execution intervals for respective variables(y_data), timestamps(x_data, indexs)]



In [ ]:
to_plot

In [ ]:
for x in to_plot[7:]:   ### 7 stands for sense_temperature
    #print(len(x))
    # print(x[2][0][2])
    # print(x[2][0][1])
    print(x[0])
    for (x_ind, x_time) in zip(x[2][0][2], x[2][0][1]):
        print(x_ind, x_time)

        if x_ind > 146601:
            break

    #break

In [ ]:
import plotly.graph_objects as go
import pandas as pd


### name represents the name of respective variable with which file will be saved
for (name, log_names, xy_data) in to_plot:
    ### path to save the plots
    to_write_name = name.replace('trace_data', 'exe plots')
    file_name = os.path.basename(to_write_name)
    file_name = f'{thread_typ}_version{version}_{behaviour}_{file_name}'
    dir_name = os.path.dirname(to_write_name)
    to_write_name = os.path.join(dir_name, file_name)
    #print(to_write_name)
    isPath = os.path.exists(os.path.dirname(to_write_name)) ### check if the path exists
    ### create the folder if it does not exist
    if not isPath:
        os.makedirs(os.path.dirname(to_write_name))

    
    ########## make data frame to be able to plot ################
    df = dict()
    _y_all = [] ### to adjust y-ticks
    legend_lab = [] ### collect names of the plots only
    line_style = ['solid', 'dashed', 'dashdot', 'dotted']
    markers = ['.','o','*','+','^','x','d','h',',','H','D']

    # Create figure
    fig = go.Figure()
    # print(xy_data)
    for (num, (l,xy)) in enumerate(zip(log_names, xy_data)):
        x = xy[1]
        #x = [i-x[0] for i in x]   ### get timestamps relative to first timestamp
        y = xy[0]
        ind = xy[2]
        # print(len(x),len(y),len(ind))

        # x_ticks = [(ind[i],x[i]) for i in range(0,len(x),5) ]
        # x_ticks = [(ind[i]) for i in range(0,len(x)) ]
        # print(x_ticks)

        ### ignore all the variables that are only executed once
        if xy[1]!= []:
            #print(x,y)
            df[l]=xy
            _y_all.extend(y)
            legend_lab.append(l)
            
            # plt.plot(x, y, ls=line_style[num%4], marker=markers[num%11])
            fig.add_trace(
                go.Scatter(x=ind, y=y, name=l, mode='markers', marker=dict(size=10, color='midnightblue'))
            )
            
            # Add range slider, title, yticks, axes labels
            fig.update_layout(
                title_text=f"Execution Interval for '{os.path.basename(name)}'",
                xaxis=dict(
                    title="Number of events",
                    rangeslider=dict(visible=True),
                    type='linear',
                    # tickvals=[k for k in range(0,len(x),5)],
                    # tickvals=[k for k in range(0,len(x))],
                    # ticktext=x_ticks,
                    tickfont = dict(size = 20),
                    titlefont = dict(size = 20),
                    color='black',
                    ),
                yaxis=dict(
                    title="Execution interval (ms)",
                    # tickvals=[k for k in range(1,len(_var_list)+1)],
                    # ticktext=_var_list,
                    tickfont = dict(size = 20),
                    titlefont = dict(size = 20),
                    color='black'
                    ),
                autosize=True,
                #width=500,
                #height=600,
                plot_bgcolor='rgba(0,0,0,0)'
            )
            
            fig.update_xaxes(
                mirror=True,
                ticks='outside',
                showline=True,
                linecolor='black',
                gridcolor='lightgrey'
            )
            
            fig.update_yaxes(
                mirror=True,
                ticks='outside',
                showline=True,
                linecolor='black',
                gridcolor='lightgrey'
            )
        
    if _y_all != []:

        # style all the traces
        fig.update_traces(
            #hoverinfo="name+x+text",
            line={"width": 0.5},
            marker={"size": 8},
            mode="lines+markers",
            showlegend=True,
            )

        fig.show()

        # break

### Trial